<h3> RA response PRS prediction project </h3>

In [1]:
import numpy as np
import pandas as pd
import magenpy as mgp
from pandas_genomics.scalars import Genotype


In [8]:
chr22_geno_path = "/home/mcb/users/yueli/datasets/RA_antiTNF/TrainingData/processed/Training_chr22.gen"

df = pd.read_csv(chr22_geno_path, sep=' ', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,8114,8115,8116,8117,8118,8119,8120,8121,8122,8123
0,22,rs915674,16053624,A,G,0.02,0.23,0.75,0.02,0.23,...,0.02,0.23,0.75,0.02,0.23,0.75,0.02,0.23,0.75,NaN
1,22,rs915677,16053758,A,G,0.00,0.13,0.87,0.00,0.13,...,0.00,0.13,0.87,0.00,0.13,0.87,0.00,0.13,0.87,NaN
2,22,rs9604721,16054713,C,T,0.93,0.07,0.00,0.93,0.07,...,0.93,0.07,0.00,0.93,0.07,0.00,0.93,0.07,0.00,NaN
3,22,rs4389403,16055070,A,G,0.01,0.19,0.79,0.01,0.19,...,0.01,0.19,0.79,0.01,0.19,0.79,0.01,0.19,0.79,NaN
4,22,rs9617528,16061016,C,T,0.07,0.38,0.55,0.07,0.38,...,0.07,0.38,0.55,0.07,0.38,0.55,0.07,0.38,0.55,NaN


In [33]:
def gen_to_bed(gen_file, bed_file):
    # Read the .gen file into a pandas DataFrame
    df = pd.read_csv(gen_file, sep=' ', header=None)

    # Extract the relevant columns (e.g., chromosome, start position, end position)
    # and create a new DataFrame with the desired column names
    bed_df = pd.DataFrame()
    bed_df['chromosome'] = df[0]
    bed_df['SNP'] = df[1]
    for i in range(0,2706):   # patient ids from 0-2705
        pid = 'pid_' + str(i)
        bed_df[pid] =df.iloc[:, i*3+5].astype(str) + ', ' + df.iloc[:, i*3+6].astype(str) + ', ' + df.iloc[:, i*3+7].astype(str)

    # loop to fill in the genotype probabilities for the 2706 individuals 
    bed_df.to_csv(bed_file, sep=' ', index=False, header=False)


In [34]:
chr22_geno_path = "/home/mcb/users/yueli/datasets/RA_antiTNF/TrainingData/processed/Training_chr22.gen"
chr22_bed_try = "/home/mcb/users/yzhu439/RA_Project/LD_chr22_training/chr22_try.bed"
gen_to_bed(chr22_geno_path, chr22_bed_try)

/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if sys.path[0] == "":


In [38]:
input = pd.read_csv(chr22_bed_try, sep=' ', header=None)
input 

,0,1,2,3,4,5,6,7,8,9,...,2698,2699,2700,2701,2702,2703,2704,2705,2706,2707
0,22,rs915674,"0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75",...,"0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75"
1,22,rs915677,"0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87",...,"0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87"
2,22,rs9604721,"0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0",...,"0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0"
3,22,rs4389403,"0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79",...,"0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79"
4,22,rs9617528,"0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55",...,"0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33740,22,rs2285395,"0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.13, 0.87","0.0, 0.12, 0.87","0.0, 0.17, 0.82",...,"0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0"
33741,22,rs8136603,"0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.85, 0.15, 0.0","0.85, 0.14, 0.0","0.81, 0.18, 0.01",...,"0.97, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.99, 0.01, 0.0"
33742,22,rs3888396,"0.0, 0.19, 0.8","0.0, 0.27, 0.72","0.0, 0.2, 0.8","0.01, 0.44, 0.55","0.0, 0.08, 0.92","0.0, 0.16, 0.83","0.0, 0.17, 0.82","0.0, 0.16, 0.83",...,"0.0, 0.15, 0.85","0.0, 0.07, 0.92","0.0, 0.06, 0.94","0.0, 0.07, 0.93","0.0, 0.08, 0.92","0.01, 0.23, 0.76","0.0, 0.06, 0.94","0.0, 0.06, 0.94","0.0, 0.1, 0.89","0.0, 0.07, 0.93"
33743,22,rs2238837,"0.75, 0.23, 0.02","0.12, 0.68, 0.2","0.57, 0.36, 0.07","0.59, 0.37, 0.04","0.43, 0.5, 0.07","0.38, 0.47, 0.15","0.38, 0.47, 0.15","0.38, 0.47, 0.15",...,"0.57, 0.4, 0.03","0.15, 0.73, 0.12","0.14, 0.75, 0.11","0.08, 0.51, 0.4","0.24, 0.67, 0.09","0.28, 0.53, 0.19","0.14, 0.75, 0.11","0.14, 0.76, 0.11","0.15, 0.75, 0.11","0.14, 0.75, 0.12"


In [36]:
# example for utilizing mgp
g_sim = mgp.GWASimulator(mgp.tgp_eur_data_path(),
                         h2=0.1)
g_sim.simulate()
g_sim.to_phenotype_table()
#          FID      IID  phenotype
# 0    HG00096  HG00096   0.795651
# 1    HG00097  HG00097   0.550914
# 2    HG00099  HG00099  -0.928486
# 3    HG00100  HG00100   0.893626
# 4    HG00101  HG00101  -0.670106
# ..       ...      ...        ...
# 373  NA20815  NA20815   0.246071
# 374  NA20818  NA20818   1.821426
# 375  NA20819  NA20819  -0.457994
# 376  NA20826  NA20826   0.954208
# 377  NA20828  NA20828   0.088412
#
# [378 rows x 3 columns]


> Reading BED file...
> Generating polygenic scores...


,FID,IID,phenotype
0,HG00096,HG00096,1.141083
1,HG00097,HG00097,0.704467
2,HG00099,HG00099,-0.809097
3,HG00100,HG00100,0.325869
4,HG00101,HG00101,-0.480681
...,...,...,...
373,NA20815,NA20815,0.063035
374,NA20818,NA20818,-0.044000
375,NA20819,NA20819,0.777682
376,NA20826,NA20826,3.078439


In [41]:
print(mgp.tgp_eur_data_path())

/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/magenpy/data/1000G_eur_chr22


In [42]:
chr22_bim_try = "/home/mcb/users/yzhu439/RA_Project/LD_chr22_training/chr22_try.bim"
chr22_fam_try = "/home/mcb/users/yzhu439/RA_Project/LD_chr22_training/chr22_try.fam"

bed_df = pd.read_csv(chr22_bed_try, sep=' ', header=None)
bed_df.to_csv(chr22_bim_try, sep=' ', index=False, header=False)
bed_df.to_csv(chr22_fam_try, sep=' ', index=False, header=False)



In [59]:
out = "/home/mcb/users/yzhu439/RA_Project/LD_chr22_training/chr22_tryLD_out/"
chr22_try_path = "/home/mcb/users/yzhu439/RA_Project/LD_chr22_training"
gdl = mgp.GWADataLoader(chr22_try_path,backend='plink')
 
gdl.compute_ld(estimator='windowed',
               output_dir=out,
               window_size=3.)
# gdl.compute_ld(estimator='windowed', output_dir=out, window_size=100)

TypeError: object of type 'NoneType' has no len()

<h3> Convert clinical data .txt to .sample </h3>

In [7]:
import os
def convert_txt_to_sample(file_path):
    # Read the content of the .txt file
    with open(file_path, 'r') as txt_file:
        content = txt_file.read()

    # Change the file extension to .sample
    out_path = "/home/mcb/users/yzhu439/RA_Project/temp/DREAM_RA_Responders_PhenoCov_Full"
    new_file_path = os.path.splitext(out_path)[0] + '.sample'

    # Write the content to the .sample file
    with open(new_file_path, 'w') as sample_file:
        sample_file.write(content)

    print(f"Conversion complete. File saved as {new_file_path}")
    
path = '/home/mcb/users/yueli/datasets/RA_antiTNF/TrainingData/original/DREAM_RA_Responders_PhenoCov_Full.txt'
convert_txt_to_sample(path)

Conversion complete. File saved as /home/mcb/users/yzhu439/RA_Project/temp/DREAM_RA_Responders_PhenoCov_Full.sample
